In [2]:
from random import randint

homeUrl = "http://www.barnstormers.com/"
frameUrl = homeUrl + "cat_search.php"
siteUrl = homeUrl + "cat.php"
siteUrl = "?".join([siteUrl, "mode=search"])
print(siteUrl)

proxies_array = [
    {'host': '64.120.85.192', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Chicago Ilinois'},
    {'host': '8.29.125.84', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Columbus Ohio'},
    {'host': '173.234.204.209', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Dallas Texas'},
    {'host': '8.29.124.215', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Denver Colorado'},
    {'host': '167.160.106.72', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Houston Texas'},
    {'host': '192.171.229.113', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Irvine California'},
    {'host': '45.59.21.72', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Las Vegas Nevada'},
    {'host': '142.91.235.188', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Los Angeles California'},
    {'host': '108.62.137.20', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'New York City New York'},
    {'host': '216.107.136.72', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Oklahoma City Oklahoma'},
    {'host': '23.19.213.86', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Phoenix Arizona'},
    {'host': '23.19.36.245', 'port': '56597', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>', 'loc': 'Seattle Washington'},
]
proxy = proxies_array[randint(0, len(proxies_array)-1)]

http://www.barnstormers.com/cat.php?mode=search


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import zipfile

def get_chrome_options(proxy):
    manifest_json = """
    {
        "version": "1.0.0",
        "manifest_version": 2,
        "name": "Chrome Proxy",
        "permissions": [
            "proxy",
            "tabs",
            "unlimitedStorage",
            "storage",
            "<all_urls>",
            "webRequest",
            "webRequestBlocking"
        ],
        "background": {
            "scripts": ["background.js"]
        },
        "minimum_chrome_version":"22.0.0"
    }
    """

    background_js = """
    var config = {{
            mode: "fixed_servers",
            rules: {{
              singleProxy: {{
                scheme: "http",
                host: "{host}",
                port: parseInt({port})
              }},
              bypassList: ["foobar.com"]
            }}
          }};

    chrome.proxy.settings.set({{value: config, scope: "regular"}}, function() {{}});

    function callbackFn(details) {{
        return {{
            authCredentials: {{
                username: "{usr}",
                password: "{pwd}"
            }}
        }};
    }}

    chrome.webRequest.onAuthRequired.addListener(
                callbackFn,
                {{urls: ["<all_urls>"]}},
                ['blocking']
    );
    """.format(**proxy)


    pluginfile = 'proxy_auth_plugin.zip'

    with zipfile.ZipFile(pluginfile, 'w') as zp:
        zp.writestr("manifest.json", manifest_json)
        zp.writestr("background.js", background_js)

    co = Options()
    co.add_argument("--start-maximized")
    co.add_argument('--dns-prefetch-disable')
    #co.add_argument('--disable-extensions')
    co.add_extension(pluginfile)
    
    return(co)

In [4]:
co = get_chrome_options(proxy)
driver = webdriver.Chrome(chrome_options=co)

In [5]:
import time
from bs4 import BeautifulSoup

# set timeout information
driver.set_page_load_timeout(50)

finished = 0
fails = 0
while finished == 0 and fails < 8:
    try:
        driver.get(frameUrl)
        finished = 1
    except Exception as e:
        print(str(e))
        fails = fails + 1
        time.sleep(5)

In [6]:
siteSoup = BeautifulSoup(driver.page_source, 'lxml')
siteInputs = siteSoup.select('input[type="text"]')
for siteInput in siteInputs:
    print(siteInput['name'])

filters[Keyword]
filters[Description]
filters[Headline]
filters[PartNumber]
filters[Manufacturer]
filters[Model]
filters[Company]
filters[LastName]
filters[FirstName]
filters[Country]
filters[State]
filters[City]
filters[Zip]
filters[Phone]
filters[Email]
filters[CompanyEmail]
filters[Category]
filters[PriceMin]
filters[PriceMax]


In [7]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

try:
    #<input type="text" name="filters[Keyword]" value="" size="50">
    inputCss = 'input[type="text"]'
    
    # Wait for search to show up
    inputTag = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, inputCss))
    )
    
    #print(driver.find_element_by_css_selector(inputCss).get_attribute('outerHTML'))
    inputTag.clear()
    inputTag.send_keys('plane')
except Exception as e:
    print(str(e))

In [8]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

try:
    #<input type="checkbox" name="summary" value="1">
    checkboxCss = 'input[type="checkbox"]'
    
    # Wait for search to show up
    checkboxTag = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, checkboxCss))
    )
    
    #print(driver.find_element_by_css_selector(checkboxCss).get_attribute('outerHTML'))
    checkboxTag.click()
except Exception as e:
    print(str(e))

In [9]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

try:
    #<input type="submit" value=" Search ">
    buttonCss = 'form > center > p > input[type="submit"]'
    
    # Wait for search to show up
    buttonTag = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, buttonCss))
    )
    
    buttonTag.click()
except Exception as e:
    print(str(e))

In [10]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

def wait_for(condition_function):
    start_time = time.time()
    while time.time() < start_time + 3:
        if condition_function():
            return True
        else:
            time.sleep(0.1)
    raise Exception(
        'Timeout waiting for {}'.format(condition_function.__name__)
    )

class wait_for_page_load(object):

    def __init__(self, browser):
        self.driver = driver

    def __enter__(self):
        self.old_page = self.driver.find_element_by_tag_name('html')

    def page_has_loaded(self):
        new_page = self.driver.find_element_by_tag_name('html')
        time.sleep(0.1)
        return new_page.id != self.old_page.id

    def __exit__(self, *_):
        wait_for(self.page_has_loaded)

def get_page_urls(driver):
    page_source = driver.page_source
    current_url = driver.current_url
    
    # Retrieve the page with tag results and set it up to be scraped
    soup = BeautifulSoup(page_source, 'lxml')
    
    #<a href="ad_detail.php?ID=1212950"><font color="#000000" size="2"><b>GREAT LAKES AEROBATIC BIPLANE</b></font></a>
    pageLinks = soup.select('td.cellleft > a')
    for pageLink in pageLinks:
        pageUrl = urllib.parse.urljoin(current_url, re.sub(' ', '%20', pageLink['href']))
        if pageUrl not in seen_urls_array:
            seen_urls_array.append(pageUrl)

In [11]:
from bs4 import BeautifulSoup
import urllib.parse
import re

seen_urls_array = []
get_page_urls(driver)
while True:
    try:
        with wait_for_page_load(driver):
            #<a href="/listing.php?mode=search&amp;summary=1&amp;
            #page=2#topofpage" class="pagination next">Next Page &gt;</a>
            driver.find_element_by_css_selector('a[class="pagination next"]').click()
        get_page_urls(driver)
    except NoSuchElementException as e:
        print('The class="pagination next" link is missing: ' + str(e))
        break  # cannot click the link anymore
    except TimeoutException as e:
        print('We timed out: ' + str(e))
        break  # cannot click the link anymore
    except Exception as e:
        if re.search(r"Timeout", str(e)):
            break # cannot click the link any more
        else:
            print(str(e))

In [13]:
print(len(seen_urls_array))

695


In [12]:
import pickle

with open('barnstormers.pickle', 'wb') as handle:
    pickle.dump(seen_urls_array, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle

with open('barnstormers.pickle', 'rb') as handle:
    seen_urls_array = pickle.load(handle)

In [19]:
from random import randint
import requests

pageUrl = seen_urls_array[randint(0,len(seen_urls_array)-1)]
print(pageUrl)

for proxy in proxies_array:
    proxy_url = 'http://{usr}:{pwd}@{host}:{port}/'.format(**proxy)
    proxies = {'http': proxy_url}
    
    try:
        page = requests.get(url=pageUrl, proxies=proxies)
        soup = BeautifulSoup(page.content, 'lxml')
    except Exception as e:
        print(proxy_url, str(e))

http://www.barnstormers.com/ad_detail.php?ID=1217871
http://<<USERID>>:<<PASSWORD>>@64.120.85.192:56597/ HTTPConnectionPool(host='64.120.85.192', port=56597): Max retries exceeded with url: http://www.barnstormers.com/ad_detail.php?ID=1217871 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x000001F2784F62B0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it',)))
http://<<USERID>>:<<PASSWORD>>@8.29.125.84:56597/ HTTPConnectionPool(host='8.29.125.84', port=56597): Max retries exceeded with url: http://www.barnstormers.com/ad_detail.php?ID=1217871 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x000001F2784F69B0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused i

In [26]:
import re

#<font size="4"><a href="/classified_1217529_Cherokee+140.html" style="text-decoration:none;color:black"><b>CHEROKEE 140</b>
#</a> • $23,500 • <font color="Red"><b>FOR IMMEDIATE SALE</b></font> • </font>
price = "$Unknown"
priceCss = 'p[align="left"] > font[size="4"]'
priceString = ""
priceTag = soup.select(priceCss)
if len(priceTag):
    priceString = priceTag[0].get_text().strip()
if len(priceString):
    match = re.search(r"\$ *[0-9,]+", str(priceString))
    if match:
        price = match.group()
print(price)

phoneRe = re.compile(r"\(?[0-9]{3}\)?[-. ]?[0-9]{3}[-. ]?[0-9]{4}")
price = ""
phoneTag = priceTag[0].parent
if len(phoneTag):
    phoneString = phoneTag.get_text().strip()
if len(phoneString):
    match = phoneRe.search(phoneString)
    if match:
        phone = match.group()
print(phone)

if len(phone):
    
    #<a href="/contact_seller.php?to=64465&amp;id=1217529&amp;title=Cherokee+140&amp;return=%2Fad_manager%2Fmy_ads.php"
    #>STEPHEN SEARLE </a>
    seller = soup.select('a[href*=contact_seller]')
    if(len(seller)):
        seller = seller[0].get_text().strip()
    else:
        seller = "Unknown"
    print(seller)

    #<font face="Arial, Helvetica, sans-serif" color="#000099" size="5">Cherokee 140<br><img src="/images/1x1.gif" 
    #width="1" height="10" border="0"><br></font>
    model = soup.select('div[align="center"] > font[size="5"]')
    if(len(model)):
        model = model[0].get_text().strip()
    else:
        model = "Unknown"
    print(model)

$8,900
6183156111
Daniel Settle
2-Place Quicksilver - N number


In [20]:
driver.quit()

In [30]:
import re
import requests

phoneRe = re.compile(r"\(?[0-9]{3}\)? *[-. ]? *[0-9]{3} *[-. ]? *[0-9]{4}")
otherPhoneRe = re.compile(r"Telephone: *[+0-9 ]+")
priceCss = 'p[align="left"] > font[size="4"]'
for pageUrl in seen_urls_array:
    try:

        page = requests.get(url=pageUrl, proxies=proxies)
        soup = BeautifulSoup(page.content, 'lxml')

        price = "$Unknown"
        priceString = ""
        priceTag = soup.select(priceCss)
        if len(priceTag):
            priceString = priceTag[0].get_text().strip()
        if len(priceString):
            match = re.search(r"\$ *[0-9,]+", str(priceString))
            if match:
                price = match.group()

        phone = ""
        phoneTag = priceTag[0].parent
        if len(phoneTag):
            phoneString = phoneTag.get_text().strip()
        if len(phoneString):
            match = phoneRe.search(phoneString)
            if match:
                phone = match.group()
            else:
                match = otherPhoneRe.search(phoneString)
                if match:
                    phone = match.group().strip()

        if len(phone):

            seller = soup.select('a[href*=contact_seller]')
            if(len(seller)):
                seller = seller[0].get_text().strip()
            else:
                seller = "Unknown"

            model = soup.select('div[align="center"] > font[size="5"]')
            if(len(model)):
                model = model[0].get_text().strip()
            else:
                model = "Unknown"

            bigFile = open('barnstormers.txt', 'a', encoding='utf-8')
            bigFile.write(seller + ': ' + model + '\t' + price + '\t' + phone + '\n')
            bigFile.close()

        else:
            print(phoneTag)

    except Exception as e:
        print(str(e))

<p align="left" style="margin-bottom: 0pt;"><font size="4"><a href="/classified_1202906_NEW+SPORT+AIRCRAFT.html" style="text-decoration:none;color:black"><b>NEW SPORT AIRCRAFT</b></a>  <font color="Green"><b>PARTNER WANTED</b></font>  </font>
Developing new cheap, keep at home, car towable, fun plane. Need distribtors / business partners.
 Contact <a href="/contact_seller.php?to=60693&amp;id=1202906&amp;title=NEW+SPORT+AIRCRAFT&amp;return=%2Fad_manager%2Fmy_ads.php">Neale T. Dunstan</a> - <a href="/contact_seller.php?to=60693&amp;id=1202906&amp;title=NEW SPORT AIRCRAFT&amp;return=%2Fad_manager%2Fmy_ads.php">SERENITY AVIATION</a>, Owner - located Singapore, SI Singapore
 Telephone: 65 85746876 .
+65 85746876
 Posted February 3, 2017
<span style="font-size:8pt;">  <a href="listing.php?mode=usersearch&amp;user=60693">Show all Ads posted by this Advertiser</a>
 <a href="/recommend.php?id=1202906&amp;title=NEW+SPORT+AIRCRAFT">Recommend This Ad to a Friend</a>
 <a href="/contact_sell